### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 1 - Digital Image Processing

In this assignment you will use Python (using Pillow or OpenCV) to load, transcode and store images. You will also use the libraries to extract some basic metadata from the images and store them in a data structure for easy access.

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.


## Setup

For general installation instructions, please refer to the ressources given for all the assignments [in Moodle](https://moodle.univie.ac.at/course/view.php?id=260637#section-13).

If the cell below executes without error, you can start the assignment!

In [151]:
# -------- Imports --------
# Please do not change the contents of this cell!

# In case you work in a local environment on your own machine,
# how to install the required packages:
#   PIP:     pip install <name> / pip install --upgrade <name>
#   CONDA:   conda install -c conda-forge <name>  /  conda install -c anaconda <name>


# Imports required by us.
import cv2                             # opencv-python
from PIL import Image                  # pillow
from PIL.ExifTags import TAGS
from matplotlib import pyplot as plt   # matplotlib
from sklearn.metrics.pairwise import cosine_similarity  # scikit-learn
import numpy as np                     # numpy
import pandas as pd                    # pandas
from IPython.display import display    # packaged with python

# This directive will allow matplotlib to render interactive plots in the notebook.
%matplotlib widget

In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [152]:
# Please place your own imports here.
from PIL import ImageFilter 
import glob
from skimage import data, color, img_as_ubyte
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter

In [122]:
# Place any helper functions, global variables and classes here.

## Task 1.1: Image formats transformation and adding filters


In [106]:
# Write your function here.

# Transcodes images to JPEG format of adjustable quality (for JPEG).
def JPEGImageConverter(inputImg: str, outputDir: str, quality: float = 1.0) -> None:
    quality_factor = int(quality*100)
    img = Image.open(inputImg).convert('RGB')
    img.save(outputDir,'JPEG', quality = quality_factor)

In [107]:
# Test your function here.
JPEGImageConverter("images/the_real_firefox_dithered.bmp", "./output_dir" + ".jpeg")

In [92]:
# Add Blur filter
def BlurImage(inputImg: str) -> None:
    img = Image.open(inputImg)
    img.convert('RGB')
    img.filter(ImageFilter.BLUR).save("images/blurredimg.jpg")

In [93]:
# Test your function here.
BlurImage("images/b2seaanimals005.jpg")

## Task 1.2:  Extract / Get Image Metadata

In [133]:
# Write your function here. 

# Extracts metadata from an image and places them into a Pandas DataFrame.
def ImageMetadataGenerator(inputImage: str) -> pd.DataFrame:
    images = glob.glob(inputImage)
    df = pd.DataFrame()
    for image in images:
        with open(image, 'rb') as file:
            img = Image.open(file)
            
            # height, width
            width, height = img.size
            
            # no of channels
            channels = len(img.getbands())
            
            # bits per pixel
            mode_to_bpp = {'1':1, 'L':8, 'P':8, 'RGB':24, 'RGBA':32, 'CMYK':32, 'YCbCr':24, 'I':32, 'F':32}
            bpp = mode_to_bpp[img.mode]
            
            # no of components
            

            #colorspace type
            colorspace = img.mode

            data = np.array(([width, height, 0, channels, bpp, colorspace]))
            df2 = pd.DataFrame(data = [data],columns = ['width', 'height', 'no of components', 'no of channel', 'bits per pixel', 'colorspace type'])
            df = df.append(df2 ,ignore_index = True)    
    
    return df


In [135]:
# Test your function here.
ImageMetadataGenerator("images/Task1.3/*")

,width,height,no of components,no of channel,bits per pixel,colorspace type
0,450,298,0,1,8,L
1,450,298,0,3,24,RGB
2,433,450,0,3,24,RGB
3,450,298,0,3,24,RGB


## Task 1.3: Drawing circles around objects in an image

In [26]:
# Write your function here. 

# Identifies shapes in an images and draws circles around them.
def IdentifyObjects(inputImage: str) -> None:
    img = cv2.imread(inputImage)
    
    img_grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
    blur = cv2.GaussianBlur(img_grayscale, (5, 5),
                           cv2.BORDER_DEFAULT)
    
    ret, thresh = cv2.threshold(blur, 200, 255,
                               cv2.THRESH_BINARY_INV)
    contours, hierarchies = cv2.findContours(
        thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)


    blank = np.zeros(thresh.shape[:2],
                 dtype='uint8')
 
    #cv2.drawContours(blank, contours, -1,
     #               (255, 0, 0), 1)
   

    cv2.fillPoly(img, pts =contours, color=(0,0,0))


    for i in contours:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            #cv2.drawContours(img, [i], -1, (0, 255, 0), 2)
            cv2.circle(img, (cx, cy), 7, (0, 0, 255), -1)
            cv2.putText(img, "center", (cx - 20, cy - 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
  

            
    cv2.imwrite("images/Task1.3/modified.jpg", img)

In [27]:
# Test your function here.
IdentifyObjects('images/Task1.3/partyballoons-1.jpg')

## Task 1.4: Duplicate image finder using cosine similarity

In [357]:
# Calculates a histogram for one image.
def ImageHistogramGenerator(inputImage: str) -> np.array:
    im = cv2.imread(inputImage)
    histr = cv2.calcHist([im],[0],None,[256],[0,256])
    #print(histr)
    #plt.plot(histr)
    #plt.show()
    return histr

In [358]:
# Test your function here.
ImageHistogramGenerator('images/jawa-3.jpg')

array([[31776.],
       [ 4039.],
       [ 4690.],
       [ 5084.],
       [ 5830.],
       [ 6166.],
       [ 6520.],
       [ 6808.],
       [ 7016.],
       [ 7143.],
       [ 7154.],
       [ 7309.],
       [ 7500.],
       [ 7431.],
       [ 7434.],
       [ 7312.],
       [ 7204.],
       [ 7096.],
       [ 6776.],
       [ 6589.],
       [ 6441.],
       [ 6309.],
       [ 6043.],
       [ 5840.],
       [ 5702.],
       [ 5546.],
       [ 5311.],
       [ 4975.],
       [ 4966.],
       [ 4761.],
       [ 4563.],
       [ 4578.],
       [ 4463.],
       [ 4238.],
       [ 4139.],
       [ 3962.],
       [ 3828.],
       [ 3751.],
       [ 3630.],
       [ 3587.],
       [ 3454.],
       [ 3210.],
       [ 3135.],
       [ 3035.],
       [ 2969.],
       [ 2876.],
       [ 2720.],
       [ 2641.],
       [ 2541.],
       [ 2344.],
       [ 2309.],
       [ 2261.],
       [ 2224.],
       [ 2162.],
       [ 2047.],
       [ 1947.],
       [ 1877.],
       [ 1804.],
       [ 1797.

In [391]:
# Find duplicate images and return their similarity scores in a dataframe.
def FindDuplicateImages(inputDir1 : str, inputDir2: str, similarityThreshold: float = 1.0) -> pd.DataFrame:
    imgs1 = glob.glob(inputDir1)
    imgs2 = glob.glob(inputDir2)

    df = pd.DataFrame(columns = ["inputDir1", "inputDir2","similarityThreshold"])

    for img1 in imgs1:
        with open(img1, 'rb') as file1:
            image1 = Image.open(file1)
            hist1 = ImageHistogramGenerator(img1) 
            hist1 = hist1.flatten()
        data = []
        for img2 in imgs2:
            with open(img2, 'rb') as file2:
                image2 = Image.open(file2)
                hist2 = ImageHistogramGenerator(img2)
                hist2 = hist2.flatten()
                cos = cosine_similarity([hist1], [hist2])
                
                names = np.array([img1, img2])
                
                similarity = cos.flatten()
                
                df2 = pd.DataFrame({"inputDir1": [img1], "inputDir2": [img2], "similarityThreshold": similarity})
                df = df.append(df2 ,ignore_index = True)      
 
    return df


In [392]:
# Test your function here.
FindDuplicateImages("images/unu/*", "images/doi/*", 1.0)

,inputDir1,inputDir2,similarityThreshold
0,images/unu/b2seaanimals005.jpg,images/doi/b2seaanimals005.jpg,1.000000
1,images/unu/b2seaanimals005.jpg,images/doi/b1murene.jpg,0.249399
2,images/unu/b3_seaanimals014.jpg,images/doi/b2seaanimals005.jpg,0.411934
3,images/unu/b3_seaanimals014.jpg,images/doi/b1murene.jpg,0.550042
4,images/unu/b1murene.jpg,images/doi/b2seaanimals005.jpg,0.249399
5,images/unu/b1murene.jpg,images/doi/b1murene.jpg,1.000000
